In [5]:
import sys
import os

# Define the path to your folder
folder_path = f'..\\..\\..\\Models\\AutoDiffusion' 

# Add the folder to sys.path
sys.path.append(folder_path)

In [6]:
import numpy as np
#import process_edited as pce
import process_GQ as pce
import autoencoder as ae
import diffusion as diff
import TabDDPMdiff as TabDiff
import pandas as pd
import torch
import os
import time
import numpy as np

In [7]:
# Function to adjust the correlations of the generated data
def adjust_correlation(X, target_corr_matrix):
    L = np.linalg.cholesky(target_corr_matrix)
    X_centered = X - np.mean(X, axis=0)
    X_transformed = np.dot(X_centered, L.T)
    return X_transformed + np.mean(X, axis=0)



In [ ]:
# strings_set = {'diabetes','oil','yeast_ml8_dataset','creditcard_sampled','HTRU','mammography'}
Model = 'AutoDiff'
dataset = 'diabetes'
metrics_list = []

print(f"Result Metrics for AutoDiff Autoencoder & ForestDIffusion for {dataset} dataset")
file_path = f'..\\..\\..\\Datasets\\Original Data\\{dataset}.csv'
# Read dataframe
# print(file_path)
real_df = pd.read_csv(file_path)
#real_df = real_df.drop('url', axis=1)
# # Step 2: Inspect the data and check for class imbalance
# # Assuming the last column is the label, and the rest are features
X = real_df.iloc[:, :-1].values  # Features
y = real_df.iloc[:, -1].values  # Labels (binary classification)
#  # Separate the minority class
# Find the minority class

real_minortiy = real_df[y == 1]

threshold = 0.01 # Threshold for mixed-type variables
parser = pce.DataFrameParser().fit(real_minortiy, threshold)
################################################################################################################
# Auto-encoder hyper-parameters
device = 'cuda' #@param {'type':'string'}
n_epochs = 2000 #@param {'type':'integer'}
eps = 1e-5 #@param {type:"number"}
weight_decay = 1e-6 #@param {'type':'number'}
maximum_learning_rate = 1e-2 #@param {'type':'number'}
lr = 2e-4 #@param {'type':'number'}
hidden_size = 250
num_layers = 3
batch_size = real_minortiy.shape[0] # Full batch

ds = ae.train_autoencoder(real_minortiy, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold)
latent_features = ds[1].detach()

from ForestDiffusion import ForestDiffusionModel


# Convert to NumPy array
array = latent_features.detach().cpu().numpy()

correlation_matrix = pd.DataFrame(array).corr()
forest_model = ForestDiffusionModel(array, label_y=None, n_t=50, duplicate_K=100, bin_indexes=[], cat_indexes=[], int_indexes=[], diffusion_type='flow', n_jobs=-1)
minority_fake = forest_model.generate(batch_size=len(real_minortiy)) # Adjust the batch size to create a balanced dataset


# Step 6: Adjust the synthetic data to match the original data's correlation matrix
X_minority_fake_adjusted = adjust_correlation(minority_fake, correlation_matrix)


sample=torch.tensor(X_minority_fake_adjusted, dtype=torch.float32)
sample.shape
gen_output = ds[0](sample, ds[2], ds[3])
gen_df = pce.convert_to_table(real_minortiy, gen_output, threshold)

output_directory =  f'..\\..\\..\\Datasets\\Synthetic Data\\'
filename = f'{Model}+Forest_{dataset}_Synthetic.csv'
output_file = os.path.join(output_directory, filename)
gen_df.to_csv(output_file, index=False) 


# Select a random sample of the generated data
selected_samples = gen_df.sample(n=min(100,gen_df.shape[0]), random_state=42)  # For reproducibility
# Syn _df will be the dataset after augmentation
syn_df = pd.concat([real_df, selected_samples], ignore_index=True)


augmented_output_directory =  f'..\\..\\..\\Datasets\\Augmented Data\\'
filename = f'{Model}+Forest_{dataset}_Augmented.csv'
augmented_output_file = os.path.join(augmented_output_directory, filename)
syn_df.to_csv(augmented_output_file, index=False) 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from ForestDiffusion import ForestDiffusionModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, f1_score

# real_df = pd.read_csv(filename)
# syn_filename = f'{string}/{Model}_{string}_Augmented.csv'

# augmented_df = pd.read_csv(syn_filename)
augmented_df=syn_df

X = real_df.iloc[:, :-1].values  # Features
y = real_df.iloc[:, -1].values 
# Check and print the original class distribution
unique, counts = np.unique(y, return_counts=True)
class_dist_before = dict(zip(unique, counts))
print(f"Class distribution before augmentation: {class_dist_before}")# Labels (binary classification)

X_balanced = augmented_df.iloc[:, :-1].values  # Features
y_balanced = augmented_df.iloc[:, -1].values  # Labels (binary classification)

# Check and print the Augmented class distribution
unique, counts = np.unique(y_balanced, return_counts=True)
class_dist_after = dict(zip(unique, counts))
print(f"Class distribution after augmentation: {class_dist_after}")

# Step 6: Split the dataset into training and test sets (original and balanced)
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# Step 7: Train a simple classifier on both original and generated datasets
clf_orig = RandomForestClassifier(random_state=42)
clf_orig.fit(X_train_orig, y_train_orig)

clf_bal = RandomForestClassifier(random_state=42)
clf_bal.fit(X_train_bal, y_train_bal)

# Step 8: Predict and calculate recall and F1 scores
y_pred_orig = clf_orig.predict(X_test_orig)
y_pred_bal = clf_bal.predict(X_test_orig)

prec_orig = precision_score(y_test_orig, y_pred_orig)
prec_bal = precision_score(y_test_orig, y_pred_bal)

recall_orig = recall_score(y_test_orig, y_pred_orig)
recalls_bal = recall_score(y_test_orig, y_pred_bal)

f1_orig = f1_score(y_test_orig, y_pred_orig)
f1_bal = f1_score(y_test_orig, y_pred_bal)

# Step 9: Print and store the performance metrics
# Store metrics in a dictionary
metrics = {
    "Dataset": dataset,
    "Precision_Original": prec_orig,
    "Precision_Generated": prec_bal,
    "Recall_Original": recall_orig,
    "Recall_Generated": recalls_bal,
    "F1_Original": f1_orig,   
    "F1_Generated": f1_bal,
    "Num_Fake_Samples": len(augmented_df) - len(real_df),s
    "Synthetic/Original_Ratio":100*(len(augmented_df) - len(real_df))/len(real_minortiy)
}

# Append the dictionary to the list
metrics_list.append(metrics)

print(f"Precision score (original data): {prec_orig:.4f}")
print(f"Precision score (generated data): {prec_bal:.4f}")
print(f"Recall score (original data): {recall_orig:.4f}")
print(f"Recall score (generated data): {recalls_bal:.4f}")
print(f"F1 score (original data): {f1_orig:.4f}")
print(f"F1 score (generated data): {f1_bal:.4f}")
print("Classification Report (original data):\n", classification_report(y_test_orig, y_pred_orig))
print("Classification Report (generated data):\n", classification_report(y_test_orig, y_pred_bal))


print(f"Number of fake samples generated: {len(augmented_df)-len(real_df)}")

Result Metrics for AutoDiff Autoencoder & ForestDIffusion for diabetes dataset


  0%|          | 0/2000 [00:00<?, ?it/s]

Class distribution before augmentation: {0: 500, 1: 268}
Class distribution after augmentation: {0.0: 500, 1.0: 368}
Precision score (original data): 0.6386
Precision score (generated data): 0.7500
Recall score (original data): 0.6625
Recall score (generated data): 0.8250
F1 score (original data): 0.6503
F1 score (generated data): 0.7857
Classification Report (original data):
               precision    recall  f1-score   support

           0       0.82      0.80      0.81       151
           1       0.64      0.66      0.65        80

    accuracy                           0.75       231
   macro avg       0.73      0.73      0.73       231
weighted avg       0.76      0.75      0.75       231

Classification Report (generated data):
               precision    recall  f1-score   support

           0       0.90      0.85      0.88       151
           1       0.75      0.82      0.79        80

    accuracy                           0.84       231
   macro avg       0.83      0.84 

In [12]:
metrics_df = pd.DataFrame(metrics_list)
metrics_df.head()

,Dataset,Precision_Original,Recall_Original,F1_Original,Precision_Generated,Recall_Generated,F1_Generated,Num_Fake_Samples
0,diabetes,0.638554,0.6625,0.650307,0.75,0.825,0.785714,100
